In [1]:
!pip install -U sentence-transformers -qq
!pip install pandas -qq
!pip install lshashpy3 -qq
!pip install numpy -qq
!pip install datasketch -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 2.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
df = pd.read_csv("linkedInjobs_event_new.csv")

df.drop_duplicates(subset='Job ID', keep='first', inplace=True)

# https://ekzhu.com/datasketch/documentation.html

In [3]:
from sentence_transformers import SentenceTransformer
from datasketch import MinHashLSH, MinHash, MinHashLSHEnsemble

# Load pre-trained sentence transformer model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Function to generate MinHash from embeddings
def generate_minhash(embedding):
    m = MinHash(num_perm=256)
    for val in embedding:
        m.update(str(val).encode('utf-8'))
    return m

df.columns = df.columns.str.strip()



/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
%%time
def initialize_lsh(df):
    lsh = MinHashLSH(threshold=0.8, num_perm=256)

    for index, row in df.iterrows():
        job_id = row['Job ID']
        company = row['Company']
        title = row['Title'].lower()
        minhash = generate_minhash(title)
        lsh.insert("job_id = {}, company = {}, title = {}".format( job_id, company, title), minhash)

    return lsh
lsh = initialize_lsh(df)


CPU times: user 41 s, sys: 317 ms, total: 41.4 s
Wall time: 41.8 s


In [5]:
def query_lsh_by_title(lsh, title):
    minhash = generate_minhash(title.lower())
    results = []
    for key in lsh.query(minhash):
        results.append(key)
    for result in results:
        print(result)
    print("Total matches:", len(results))
query_lsh_by_title(lsh, 'data analyst')


job_id = 3831344612, company = Kutir Corporation, title = data analyst
job_id = 3834996019, company = MedScout, title = data analyst
job_id = 3829280357, company = SPS Health, title = data analyst
job_id = 3836471552, company = Allied OneSource, title = data quality assistant
job_id = 3834669130, company = Independence Pet Group, title = data analyst
job_id = 3826245760, company = CNA Corporation, title = data analyst i
job_id = 3826590464, company = Media.Monks, title = data analyst
job_id = 3832107376, company = Genpact, title = data analyst
job_id = 3828197721, company = CKE Restaurants, Inc., title = data analyst
job_id = 3826751305, company = Numero Data, title = data analyst
job_id = 3834480891, company = KMRG, LLC, title = data analyst
job_id = 3836419997, company = Marathon Oil Corporation, title = natural gas analyst
job_id = 3830924923, company = Peak Gardens, title = data analyst
job_id = 3835128465, company = Synapri, title = data analyst
job_id = 3830926864, company = Peak

In [6]:
def count_titles_with_words(lsh, title):
    title = title.lower()
    words_to_check = title.split()
    minhash = generate_minhash(title)
    count_containing_words = 0
    count = 0
    for title_line in lsh.query(minhash):
        count += 1
        extracted_title = title_line.split(", title = ")[1].split()
        if any(word in extracted_title for word in words_to_check):
            count_containing_words += 1
    print(f"Number of titles containing words from '{title}': {count_containing_words}, total titles queried: {count}")
count_titles_with_words(lsh, 'data analyst')


Number of titles containing words from 'data analyst': 32, total titles queried: 32
